In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from dtaidistance import dtw
import ta


In [2]:
class Backtest:
    def __init__(self, csv :str, start :int, interval :int, port) -> None:
        self.df = pd.read_csv(csv, delimiter=',')
        self.start_idx = self.current_idx = self.df.index[self.df['unix'] == start][0]
        self.interval = int(interval / 3600)
        self.port = port
        self.owned = 0

    def get_close(self, datapoints :int):
        return self.df.iloc[self.start_idx:self.start_idx + self.interval * datapoints:self.interval]['close']
    
    def get_price(self):
        return self.df.iloc[self.current_idx]['open']

    def buy(self, volume: float):
            self.owned += volume
            self.port -= volume * self.get_price()

    def sell(self, volume: float):
        if self.owned >= volume:
            self.port += volume * self.get_price()
            self.owned -= volume
        else:
            self.port = self.owned * self.get_price()
            self.owned = 0

    def simple_rsi(self, datapoints :int, lookback=24):
        changes = self.get_close(datapoints).pct_change()
        simple_rsis = [np.nan]*lookback

        for window in changes.rolling(window=lookback):
    
            if len(window) != lookback: continue

            positives = window[window>0].sum()
            negatives = window[window<0].sum() * -1

            simple_rsis.append(100 - 100/(1 + positives/negatives))

        return pd.Series(index = self.get_close(datapoints).index, data=simple_rsis[:-1])
    
    def get_signals(self, datapoints, buy_thresh: int, sell_thresh: int):
        rsi_data = self.simple_rsi(datapoints)
        print(f'length of rsi_data is {len(rsi_data)}')
        actions = []
        for rsi in rsi_data:
            if rsi == np.nan: actions.append(None)
            
            if rsi >= sell_thresh: actions.append('SELL')
            elif rsi <= buy_thresh: actions.append('BUY')
            else: actions.append(None)
        print(f'length of actions is {len(actions)}')
        return pd.Series(index=rsi_data.index, data=actions)

    def calc_pnl(self):
        return self.port + self.owned * self.get_price()

In [3]:
def run_algo():
    ETHUSD = Backtest('BacktestData.csv', 1670396400, 3600*4, 10000)
    signals = ETHUSD.get_signals(2000, 35, 65)
    k = []
    for signal in signals:
        if signal == 'BUY' :
            ETHUSD.buy(ETHUSD.port/ETHUSD.get_price())
        elif signal == 'SELL':
            ETHUSD.sell(ETHUSD.owned)
        else:
            pass
        ETHUSD.current_idx += ETHUSD.interval
        k.append(ETHUSD.calc_pnl())
    print(max(k))
    print(ETHUSD.calc_pnl())

In [47]:
run_algo()

NameError: name 'run_algo' is not defined

In [ ]:
# data labeling
# write + implement dtw
# implement kmeans clustering (+ heirarchical clustering as a side bonus?)
# cut out outliers from the dataset
# create the ml model using indicators (see paper)
# train the model (80/20 train test split)
# signal generation + final implementation + paper live test?
# + finding optimal k

In [2]:
class KMeansDTW():
    def __init__(self, k: int = 8, max_iter: int = 3000, tol: float = 0.001):
        self.k = k
        self.max_iter = max_iter
        self.tol = tol
        
    def create_clusters(self, data: np.ndarray):
        # Initialize centroids randomly
        rand_idx = np.random.choice(data.shape[0], self.k, replace=False)
        self.centroids = data[rand_idx]
        
        for _ in tqdm(range(self.max_iter)):
            self.classifications = [[] for _ in range(self.k)]
            
            # Precompute distances between each datapoint and each centroid
            # This step is assumed to be the optimized part; depending on the fastdtw implementation details
            # You might need to manually loop through data and centroids if fastdtw cannot be vectorized directly
            for i, datapoint in enumerate(data):
                distances = np.array([dtw.distance_fast(centroid, datapoint) for centroid in self.centroids])
                closest_centroid_idx = np.argmin(distances)
                self.classifications[closest_centroid_idx].append(datapoint)
            
            prev_centroids = np.copy(self.centroids)
            for i, classification in enumerate(self.classifications):
                # Efficiently compute new centroids
                if classification:  # Check if classification is not empty
                    self.centroids[i] = np.mean(classification, axis=0)
            
            # Check for convergence
            optimised_flag = True
            for i in range(self.k):
                diff = np.linalg.norm(prev_centroids[i] - self.centroids[i])
                if diff >= self.tol:
                    optimised_flag = False
                    break
            
            if optimised_flag:
                break

    def elbow_method(self):
        total_var = 0
        for i, centroid in enumerate(self.centroids):
            for datapoint in self.classifications[i]:
                total_var += dtw.distance_fast(centroid, datapoint)
        return total_var
    
    def display_clusters(self):
        for i, cluster in enumerate(self.classifications, start = 1):
            plt.figure(figsize=(3, 1.5))
            for series in cluster:
                plt.plot(series)

        plt.title(f'Cluster {i} Time Series')
        plt.show()



In [3]:
my_KMeans = KMeansDTW()

In [108]:
my_KMeans.create_clusters(tester)
print(my_KMeans.classifications)

<class 'numpy.ndarray'>
[203 400 274]
[[-0.9924995  -0.99150183 -0.98640069 -0.97765588 -0.97927818 -0.97677967
  -0.98046672 -0.97646735 -0.97794217 -0.9796946  -0.97722211 -0.98251411
  -0.98142968 -0.98086578 -0.98156849 -0.98000692 -0.97939097 -0.97811568
  -0.97616372 -0.97378665 -0.97752575 -0.97228581 -0.97290176 -0.97326613
  -0.97637192 -0.97501856 -0.96800884 -0.97102788 -0.96825175 -0.9687983
  -0.96983067 -0.97022974 -0.97915673 -0.9791307  -0.97782072 -0.97848005]
 [ 1.57500343  1.58638555  1.67893476  1.68174559  1.72048994  1.69703165
   1.69608603  1.69127985  1.67959409  1.6902041   1.68434821  1.68044428
   1.65086983  1.65838273  1.63270353  1.66095065  1.68825214  1.65205836
   1.62945893  1.6491174   1.63053468  1.76366742  1.72786403  1.67189033
   1.68414     1.77134515  1.80802476  1.84202366  1.83946442  1.84806174
   1.93428657  1.95412722  1.94246748  2.08698235  2.14286062  2.12751383]
 [-0.75553086 -0.75298897 -0.75094157 -0.73772893 -0.7401407  -0.73982838

  0%|          | 0/3000 [00:13<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (234,) (234,36) 

In [7]:
class HierarchDTW():
    # inits a df but operates on [subseries]
    def __init__(self, df: pd.DataFrame):
        self.df= df
        self.linkages_matrix = None
    
    def split_time_series(self, window: int, slide: int):
        self.data = []
        series = zscore(self.df['close'])
        for i in range(0, len(series) - window, slide):
            self.data.append(list(series.iloc[i:i+window]))
        self.data = np.array(self.data)

    def calc_dist_matrix(self):
        num_datapoints = self.data.shape[0]
        self.distance_matrix = np.zeros((num_datapoints, num_datapoints))
        for i in range(num_datapoints):
            for j in range(i + 1, num_datapoints):
                self.distance_matrix[i][j] = self.distance_matrix[j][i] = dtw.distance_fast(self.data[i], self.data[j])
        
    def cluster(self, method = 'ward'):
        if self.linkages_matrix is None:
            self.split_time_series(window = 36, slide = 18)
            print(type(self.data), type(self.data[0]))
            self.calc_dist_matrix()
        self.linkages_matrix = linkage(squareform(self.distance_matrix), method = method)

    def plot_dendrogram(self):
        if self.linkages_matrix is None:
            print('not clustered yet')
        else:
            plt.figure(figsize = (10, 7))
            dendrogram(self.linkages_matrix)
            plt.xlabel('Sample index')
            plt.ylabel('Distance')
            plt.show()

    def get_clusters(self, max_clusters, criterion = 'maxclust'):
        return fcluster(self.linkages_matrix, t = max_clusters, criterion = criterion)
        

    def display_clusters(self, max_clusters: int):
        print(self.linkages_matrix.shape)
        cluster_labels = self.get_clusters(max_clusters)
        # print(cluster_labels, type(cluster_labels))
        for i in range(1,  len(np.unique(cluster_labels)) + 1):
            idx = np.where(cluster_labels == i)
            plt.plot(self.data[idx])
            plt.title(f'Cluster {i} for hierarchical')
            plt.show()
        
    def find_outliers_idx(self, max_clusters: int, cluster_nums : [int]):
        # assumes that 
        idx = np.array([])
        cluster_labels = fcluster(self.linkages_matrix, t = max_clusters, criterion = 'maxclust')
        for i in cluster_nums:
            idx = np.append(idx, np.where(cluster_labels == i))
        # print(idx, type(idx), type(idx[0]))
        return idx.astype(int)
    
    def return_cleaned_data(self, max_clusters, cluster_nums):
        outliers = self.find_outliers_idx(max_clusters = max_clusters, cluster_nums = cluster_nums)
        return np.delete(self.data, outliers, axis = 0)
            
    

In [ ]:
# on visual inspection, cluster 6, 7 and 12 look to be outliers so i will cut them outj

In [11]:
class Indicators():
    def __init__(self, df):
        self.df = df

    def calculate_roc(self, subseries):
        roc_indicator = ta.momentum.ROCIndicator(close=subseries['close'], window = len(subseries) - 1).roc()
        return roc_indicator.iloc[-1]

    def calculate_rsi(self, subseries):
        rsi_indicator = ta.momentum.RSIIndicator(close=subseries['close'], window = len(subseries)).rsi()
        return rsi_indicator.iloc[-1]

    def calculate_ema(self, subseries):
        ema_indicator = ta.trend.EMAIndicator(close=subseries['close'], window = len(subseries)).ema_indicator()
        return ema_indicator.iloc[-1]

    def calculate_macd(self, subseries):
        macd = ta.trend.MACD(close=subseries['close'], window_slow = 26, window_fast = 12, window_sign = 9)
        macd_indicator = macd.macd()
        macd_signal = macd.macd_signal()
        macd_diff = macd.macd_diff()
        return macd_indicator.iloc[-1], macd_signal.iloc[-1], macd_diff.iloc[-1]

    def calculate_cci(self, subseries):
        cci_indicator = ta.trend.CCIIndicator(high=subseries['high'], low=subseries['low'], close=subseries['close'], window = len(subseries)).cci()
        return cci_indicator.iloc[-1]

    def calculate_bollinger(self, subseries):
        bollinger = ta.volatility.BollingerBands(close=subseries['close'], window = len(subseries))
        bollinger_mavg = bollinger.bollinger_mavg()
        bollinger_hband = bollinger.bollinger_hband()
        bollinger_lband = bollinger.bollinger_lband()
        return bollinger_mavg.iloc[-1], bollinger_hband.iloc[-1], bollinger_lband.iloc[-1]

    def calculate_stochastic(self, subseries):
        stoch = ta.momentum.StochasticOscillator(high=subseries['high'], low=subseries['low'], close=subseries['close'])
        stoch_stoch = stoch.stoch()
        stoch_signal = stoch.stoch_signal()
        return stoch_stoch.iloc[-1], stoch_signal.iloc[-1]

    def get_featureset(self, window):
        featureset = []
        for i in range(0, len(self.df['close']) - window, int(window/2)):
            subseries = self.df.iloc[i:i + window]
            features = {
                'ROC': self.calculate_roc(subseries),
                'RSI': self.calculate_rsi(subseries),
                'EMA': self.calculate_ema(subseries),
                # For MACD, signal, and diff, handle multiple return values
                'MACD': self.calculate_macd(subseries)[0],
                'MACD_signal': self.calculate_macd(subseries)[1],
                'MACD_diff': self.calculate_macd(subseries)[2],
                'CCI': self.calculate_cci(subseries),
                # Bollinger Bands
                'Bollinger_Mavg': self.calculate_bollinger(subseries)[0],
                'Bollinger_hband': self.calculate_bollinger(subseries)[1],
                'Bollinger_lband': self.calculate_bollinger(subseries)[2],
                'Stoch_%K': self.calculate_stochastic(subseries)[0],
                'Stoch_%D': self.calculate_stochastic(subseries)[1],
            }
            featureset.append(features)
        
        return pd.DataFrame(featureset)
    
    # def slice_idx(self, window, slide):
    #     slice_idx = []
    #     series = zscore(self.df['close'])
    #     for i in range(0, len(series) - window, slide):
    #         slice_idx.append(range(i, i + window))
    #     self.data = np.array(slice_idx)
    
    def return_labels(self, window, thresh):
        labels = []
        for i in range(0, len(self.df['close']) - window, int(window/2)):
            current_price = self.df['close'].iloc[i]
            future_price = self.df['close'].iloc[i + window]
            pct_change = (future_price - current_price) / current_price * 100
            # Determine label based on threshold
            if pct_change >= thresh: labels.append(1)
            elif pct_change <= -thresh: labels.append(-1)
            else: labels.append(0)

        return labels

In [12]:
train_df = pd.read_csv('TrainData.csv', delimiter=',')
my_Hierarch = HierarchDTW(train_df)
my_Hierarch.cluster()

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [13]:
window = 36
slide = 18
indicators = Indicators(train_df)
X_uncleaned = indicators.get_featureset(window)
Y_uncleaned = indicators.return_labels(thresh = 0.015, window = window)

idx_outliers = my_Hierarch.find_outliers_idx(max_clusters = 12, cluster_nums = [6, 7, 12])

X_cleaned = X_uncleaned.drop(idx_outliers)
Y_cleaned = [subseries for i, subseries in enumerate(Y_uncleaned) if i not in idx_outliers]


In [14]:
print(X_uncleaned)

           ROC        RSI          EMA       MACD  MACD_signal  MACD_diff  \
0    -3.066849  49.792829   704.398068   3.215764     0.016586   3.199178   
1   -11.654366  32.200984   661.444919 -26.389482   -15.822083 -10.567399   
2   -18.477442  35.554372   620.317108 -15.357053   -19.332971   3.975918   
3    -4.265466  49.254646   565.182666  -7.718505   -13.108627   5.390122   
4    -0.607887  52.829282   567.738559   4.936302     3.968749   0.967554   
..         ...        ...          ...        ...          ...        ...   
684   7.026760  61.574883  2304.930564   8.272684     0.738220   7.534465   
685  -2.643573  43.780738  2291.895486 -18.537858   -13.702638  -4.835220   
686  -8.058747  35.022571  2266.157579 -19.488237   -19.019400  -0.468837   
687  14.390147  74.074173  2324.648654  64.524922    37.793686  26.731236   
688  15.663957  60.782539  2483.753710  41.962057    59.102522 -17.140466   

            CCI  Bollinger_Mavg  Bollinger_hband  Bollinger_lband   Stoch_%

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X_cleaned, Y_cleaned, test_size = 0.3, shuffle = False)
print(len(X_train), len(X_test), len(Y_train), len(Y_test))

429 185 429 185


In [19]:
clf = RandomForestClassifier(n_estimators=100, random_state=69)
clf.fit(X_train, Y_train)

predictions = clf.predict(X_test)

accuracy = accuracy_score(Y_test, predictions)

accuracy = accuracy_score(Y_test, predictions)
print(accuracy)

0.9783783783783784
